In [1]:
!free -g

              total        used        free      shared  buff/cache   available
Mem:            251           8         129           0         113         241
Swap:             1           0           1


In [2]:

# !pip install transformers==2.5.0 -q 


In [3]:
!nvidia-smi

Sat Mar 20 18:50:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 455.23.05    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  PH402 SKU 200       On   | 00000000:1D:00.0 Off |                    0 |
| N/A   58C    P0    43W / 140W |      6MiB / 32630MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  PH402 SKU 200       On   | 00000000:1E:00.0 Off |                    0 |
| N/A   

In [4]:
#  #colab setting 
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/My Drive/Competitions/医学影像报告异常检测/code')

In [5]:
import torch
import transformers
from transformers import AdamW,get_linear_schedule_with_warmup
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [6]:
# !mkdir chinese-macbert-base


# !wget https://huggingface.co/hfl/chinese-macbert-base/resolve/main/config.json  -P  chinese-macbert-base/

# !wget https://huggingface.co/hfl/chinese-macbert-base/resolve/main/added_tokens.json  -P   chinese-macbert-base/

# !wget https://huggingface.co/hfl/chinese-macbert-base/resolve/main/pytorch_model.bin  -P  chinese-macbert-base/
# !wget https://huggingface.co/hfl/chinese-macbert-base/resolve/main/special_tokens_map.json    -P chinese-macbert-base/

# !wget https://huggingface.co/hfl/chinese-macbert-base/resolve/main/tokenizer_config.json  -P  chinese-macbert-base/

# !wget https://huggingface.co/hfl/chinese-macbert-base/resolve/main/vocab.txt -P chinese-macbert-base/




In [7]:
train_path = "../input/track1_round1_train_20210222.csv"
test_path = "../input/track1_round1_testA_20210222.csv"
train_df = pd.read_csv(train_path,sep=',',header=None)
test_df = pd.read_csv(test_path,sep=',',header=None)

train_df[1]=train_df[1].apply(lambda x:x[1:-1])
train_df[2]=train_df[2].apply(lambda x:x[1:])

test_df[1] = test_df[1].apply(lambda x:x[1:-1])


In [8]:
print(len(train_df))
train_df.drop_duplicates(subset=[1,2],inplace=True)
print(len(train_df))

10000
10000


In [9]:
train_df.head(20)

,0,1,2
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2
1,1|,48 328 538 382 809 623 434 355 382 382 363 145...,
2,2|,623 656 293 851 636 842 698 493 338 266 369 69...,15
3,3|,48 328 380 259 439 107 380 265 172 470 290 693...,
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16
5,5|,80 328 328 54 172 439 741 380 172 842 698 177 ...,15
6,6|,48 322 795 856 374 439 48 328 443 380 597 172 ...,4 11 15
7,7|,623 328 659 486 582 162 711 289 606 405 809 78...,
8,8|,852 328 471 585 117 458 399 607 693 380 522 62...,6 12 14 15
9,9|,229 172 200 737 437 547 651 693 623 328 355 65...,1 3


In [10]:
test_df.head()

,0,1
0,0|,852 328 697 538 142 355 582 800 728 4 647 169 ...
1,1|,380 358 343 654 171 832 47 832 690 693 48 563 ...
2,2|,751 335 834 582 717 583 585 693 623 328 107 38...
3,3|,623 328 649 582 488 12 578 623 538 382 382 265...
4,4|,83 293 398 797 382 363 145 424 693 698 800 691...


In [11]:
from sklearn.utils import shuffle

In [12]:
def data_aug(df):
    
    import random
    df = df.copy()
    df[2]=df[2].apply(lambda x:x.split())
    new_df = df.copy()
    m =  shuffle(df)
    m.reset_index(drop=True,inplace=True)
    new_df[0]=100000
    new_df[1] = new_df[1]+m[1]
    new_df[2] = new_df[2]+m[2]
    new_df[2]=new_df[2].apply(lambda x:" ".join(list(set(x))))
    
    return new_df

In [13]:
m = data_aug(train_df)
m.head(20)

,0,1,2
0,100000,623 328 538 382 399 400 478 842 698 137 492 26...,8 0 7 5 2 1
1,100000,48 328 538 382 809 623 434 355 382 382 363 145...,
2,100000,623 656 293 851 636 842 698 493 338 266 369 69...,15
3,100000,48 328 380 259 439 107 380 265 172 470 290 693...,15 10
4,100000,623 328 399 698 493 338 266 14 177 415 511 647...,9 16
5,100000,80 328 328 54 172 439 741 380 172 842 698 177 ...,15 7
6,100000,48 322 795 856 374 439 48 328 443 380 597 172 ...,15 2 4 11
7,100000,623 328 659 486 582 162 711 289 606 405 809 78...,14
8,100000,852 328 471 585 117 458 399 607 693 380 522 62...,6 16 15 12 14
9,100000,229 172 200 737 437 547 651 693 623 328 355 65...,4 15 5 3 12 11 1


## config

In [14]:
# 建立分词器

import transformers

class config:
    DEVICE = "cuda"
    MAX_LEN = 200
    TRAIN_BATCH_SIZE = 30
    VALID_BATCH_SIZE = 30
    MODEL_PATH = "model.bin"
    EPOCHS = 10
    BERT_PATH = "n_gram_mask_bert_v2"
    TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)


save_path = config.BERT_PATH+'_weight_v2'

## SEED

In [15]:
import random 
import torch  
import numpy as np
import os
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)

## DataSet

In [16]:

import torch


class BERTDataset:
    def __init__(self,text1,text2, target):
        self.text1 = text1
        self.text2 = text2
        self.target = target
        self.tokenizer = config.TOKENIZER
        self.max_length = config.MAX_LEN

    def __len__(self):
        return len(self.text1)

    def get_dumm(self,s):
        # print("print s",s)
        # print("s split",s.split(' '))

        re=[0]*17
        if s=='' or s==" ":
            return re
        else:
            tmp=[int(i) for i in s.split(' ') if i!='']
            for i in tmp:
                re[i]=1
        return re


    def __getitem__(self, item):
        text1 = str(self.text1[item])
        text1 = " ".join(text1.split())
        text2 = None
        # text2 = str(self.text2[item])
        # text2 = " ".join(text2.split())

        # inputs = self.tokenizer.encode_plus(
        #     text1,
        #     text2,
        #     # add_special_tokens=True,
        #     max_length=self.max_len,
        #     pad_to_max_length=True,
        #     truncation=True
        # )

        # ids = inputs["input_ids"]
        # mask = inputs["attention_mask"]
        # token_type_ids = inputs["token_type_ids"]


        inputs = self.tokenizer.encode_plus(
            text1,
            text2,
            add_special_tokens=True,
            max_length=self.max_length
#             truncate_first_sequence=True  # We're truncating the first sequence in priority
        )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        mask_padding_with_zero=True
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = self.max_length - len(input_ids)
        pad_on_left = False
        pad_token=0
        pad_token_segment_id=0

        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            attention_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + attention_mask
            token_type_ids = ([pad_token_segment_id] * padding_length) + token_type_ids
        else:
            input_ids = input_ids + ([pad_token] * padding_length)
            attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
            token_type_ids = token_type_ids + ([pad_token_segment_id] )* padding_length
                                               
        label = self.target[item]
        # print("label",label)
        #print(label,[i for i in label])
        label=self.get_dumm(label)


        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(label, dtype=torch.float),
        }

In [17]:
from transformers import BertPreTrainedModel,BertModel
# from transformers.modeling_roberta import RobertaModel


## Model

In [18]:

import transformers
import torch.nn as nn

class CustomBert(BertPreTrainedModel):
    def __init__(self, config):
        config.output_hidden_states = True
        config.num_labels = 17

        super(CustomBert, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(p=0.2)
        self.high_dropout = nn.Dropout(p=0.5)

        n_weights = config.num_hidden_layers + 1+13
        weights_init = torch.zeros(n_weights).float()
        weights_init.data[:-1] = -3
        self.layer_weights = torch.nn.Parameter(weights_init)

        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

        self.init_weights()

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None,
                ):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids
                            )

        hidden_layers = outputs[2]
        
        add_embeds = [torch.mean(layer,axis=1) for layer in hidden_layers]

        cls_outputs = torch.stack([self.dropout(layer[:, 0, :]) for layer in hidden_layers]+add_embeds,
                                  dim=2)
        
        cls_output = (torch.softmax(self.layer_weights, dim=0) * cls_outputs).sum(-1)

        # multisample dropout (wut): https://arxiv.org/abs/1905.09788
        logits = torch.mean(torch.stack([
            self.classifier(self.high_dropout(cls_output))
            for _ in range(5)
        ], dim=0), dim=0)

        outputs = logits
        return outputs



In [19]:

class FGM():
    """
    对抗训练方法类
    """
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=0.5, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

## train units && eval units

In [20]:
import torch
import torch.nn as nn
from tqdm import tqdm





def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs.view(-1,17), targets.view(-1, 17))


def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    fgm = FGM(model)
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=token_type_ids)

        loss = loss_fn(outputs, targets)
        loss.backward()

        fgm.attack() # 在embedding上添加对抗扰动
        outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=token_type_ids)
        loss_adv = loss_fn(outputs, targets)
        # 反向传播，并在正常的grad基础上，累加对抗训练的梯度
        loss_adv.backward() 
        # 恢复embedding参数
        fgm.restore() 


        optimizer.step()
        scheduler.step()


def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [21]:

from sklearn.metrics import f1_score,accuracy_score,recall_score
from sklearn.metrics import roc_curve, precision_recall_curve


# def threshold_search(y_true, y_proba, plot=False):
#     precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
#     thresholds = np.append(thresholds, 1.001) 
#     F = 2 / (1/precision + 1/recall)
#     best_score = np.max(F)
#     best_th = thresholds[np.argmax(F)]
#     if plot:
#         plt.plot(thresholds, F, '-b')
#         plt.plot([best_th], [best_score], '*r')
#         plt.show()
#     search_result = {'threshold': best_th , 'f1': best_score}
#     return search_result 

In [22]:

test_df[2]=''
test_dataset = BERTDataset(text1=test_df[1].values,text2=None,target=test_df[2].values)

test_data_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1
    )


In [23]:
print(test_dataset.__getitem__(0))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'ids': tensor([101, 118, 105, 134, 139, 249, 137, 126, 310, 190, 594, 295, 177, 300,
        314, 186, 947, 174, 106, 118, 105, 134, 126, 112, 139, 678, 285, 413,
        373, 190, 159, 251, 122, 164, 102,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0

In [24]:
from sklearn.model_selection import KFold,GroupKFold,StratifiedKFold
from sklearn.metrics import precision_score , recall_score	, roc_auc_score

from sklearn.model_selection import KFold,GroupKFold
# gkf =  GroupKFold(n_splits=5)
# groups_by_text_id_list = train['qid'].tolist()

skf = StratifiedKFold(n_splits=5, random_state=4590, shuffle=True)



oof_df_list = []

try:
    os.mkdir(save_path)
except:
    pass 

oof_train_list = []
pred = [] 

# score

def ss_score(targets,outputs):
    
    targets = torch.tensor(targets,dtype=torch.float)
    outputs = torch.tensor(outputs,dtype=torch.float)
    loss = nn.BCEWithLogitsLoss()(outputs.view(-1,17), targets.view(-1, 17))
    score = 1- loss/17
    return loss.cpu().detach().numpy(),score


for idx,(trn_idx,val_idx) in enumerate(skf.split(train_df,train_df[2])):
    

#     if idx<1:
#         continue
    print("开始第 %d 折训练"%(idx+1))
    

    
    model_path = "%s/bert_wwm_model_%d.bin"%(save_path,idx)
    config.MODEL_PATH = model_path
    tr_df = train_df.iloc[trn_idx]
    va_df = train_df.iloc[val_idx]
    
    tr_df.reset_index(drop=True,inplace=True)
    
    #data aug
    
#     aug_tr_df = data_aug(tr_df)
    
#     print(tr_df.shape)
    
#     tr_df = pd.concat([tr_df,aug_tr_df])
    
#     print(tr_df.shape)

    


    train_dataset = BERTDataset(
        text1=tr_df[1].tolist(),text2=None,target=tr_df[2].tolist()
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.TRAIN_BATCH_SIZE, num_workers=4,shuffle=True
    )

    valid_dataset = BERTDataset(
     text1= va_df[1].tolist(),text2=None,target=va_df[2].tolist()
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1
    )

    device = torch.device(config.DEVICE)
    model = CustomBert.from_pretrained(config.BERT_PATH,num_labels=1)
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(tr_df) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    warmup_steps = int(num_train_steps*0.1)
    optimizer = AdamW(optimizer_parameters, lr=2e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,num_warmup_steps=warmup_steps,num_training_steps=num_train_steps
    )

    min_loss = 100000
    best_score =0
    
    for epoch in range(config.EPOCHS):
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
#         loss= torch.nn.BCEWithLogitsLoss()(torch.tensor(outputs),torch.tensor(targets))
        loss,score = ss_score(targets,outputs)

        # result =  threshold_search(targets,outputs)
        # thr = result['threshold']
        # f1 = result['f1']
        # outputs = (outputs>thr).astype(int)
        # f1 = f1_score(targets,outputs)
        # precision = precision_score(targets, outputs)
        # recall =  recall_score(targets,outputs)

        print(f"loss = {loss}, score is {score}")


#         if loss<min_loss:
#             torch.save(model.state_dict(), config.MODEL_PATH)
#             min_loss = loss

        if score>best_score:
            best_score = score
            torch.save(model.state_dict(),config.MODEL_PATH)




    model.load_state_dict(torch.load( config.MODEL_PATH))
    outputs, targets = eval_fn(valid_data_loader, model, device)
    va_df['prob'] = outputs
    oof_train_list.append(va_df)

    outputs, targets = eval_fn(test_data_loader, model, device)
    pred.append(np.array(outputs))



/home/root1/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


开始第 1 折训练


Some weights of the model checkpoint at n_gram_mask_bert_v2 were not used when initializing CustomBert: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing CustomBert from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBert from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBert were not initialized from the model checkpoint at n_gram_mask_bert_v2 and are newly initialized: ['layer_weights', 'classif

loss = 0.720056414604187, score is 0.9576437473297119


100%|██████████| 69/69 [00:04<00:00, 16.68it/s]


...

100%|██████████| 66/66 [00:03<00:00, 16.57it/s]


loss = 0.6728211641311646, score is 0.9604222774505615


100%|██████████| 66/66 [00:03<00:00, 16.61it/s]


loss = 0.6726914644241333, score is 0.9604299068450928


100%|██████████| 100/100 [00:05<00:00, 16.69it/s]


In [25]:
oof_train = pd.concat(oof_train_list)
oof_train.head()

,0,1,2,prob
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[0.0073624164797365665, 0.009764820337295532, ..."
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[0.0066406819969415665, 0.012338452972471714, ..."
5,5|,80 328 328 54 172 439 741 380 172 842 698 177 ...,15,"[0.006021423730999231, 0.00625133141875267, 0...."
7,7|,623 328 659 486 582 162 711 289 606 405 809 78...,,"[0.003780707949772477, 0.0022696598898619413, ..."
14,14|,623 328 697 661 809 48 46 355 661 414 852 328 ...,0 1 8 10,"[0.9012470245361328, 0.9200030565261841, 0.138..."


In [26]:
sub = np.average(pred, axis=0) 


In [27]:
m = sub.tolist()

In [28]:
def get_dumm(s):
    re=[0]*17
    if s=='' or s==" ":
        return re
    else:
        tmp=[int(i) for i in s.split(' ') if i!='']
        for i in tmp:
            re[i]=1
    return re

oof_train['label'] = oof_train[2].apply(lambda x:get_dumm(x))

In [29]:
oof_train

,0,1,2,prob,label
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[0.0073624164797365665, 0.009764820337295532, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[0.0066406819969415665, 0.012338452972471714, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,5|,80 328 328 54 172 439 741 380 172 842 698 177 ...,15,"[0.006021423730999231, 0.00625133141875267, 0....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,7|,623 328 659 486 582 162 711 289 606 405 809 78...,,"[0.003780707949772477, 0.0022696598898619413, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14,14|,623 328 697 661 809 48 46 355 661 414 852 328 ...,0 1 8 10,"[0.9012470245361328, 0.9200030565261841, 0.138...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
21,21|,380 172 200 791 470 753 693 256 514 569 231 11...,,"[0.0018210123525932431, 0.003105448791757226, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27,27|,852 328 501 355 360 265 478 162 498 289 169 40...,15,"[0.006496835965663195, 0.008982159197330475, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
31,31|,91 380 488 12 591 487 197 852 328 538 501 382 ...,,"[0.0014628305798396468, 0.0017630878137424588,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
35,35|,380 315 809 623 328 160 380 419 789 439 852 81...,6 12 14 15,"[0.02823447249829769, 0.03760896250605583, 0.0...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, ..."
41,41|,48 718 328 419 571 769 256 100 809 328 380 172...,4 5 11 12 15,"[0.034585777670145035, 0.02986907958984375, 0....","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."


In [30]:
oof_train['score'] = oof_train.apply(lambda x:ss_score(x.label,x.prob)[1],axis=1)


In [31]:
oof_score = oof_train.score.mean()
print(oof_score)

0.96040595703125


In [32]:
test_df[2] = m
test_df.head()

,0,1,2
0,0|,852 328 697 538 142 355 582 800 728 4 647 169 ...,"[0.012090771738439799, 0.005923829087987542, 0..."
1,1|,380 358 343 654 171 832 47 832 690 693 48 563 ...,"[0.0017855744576081634, 0.002484714472666383, ..."
2,2|,751 335 834 582 717 583 585 693 623 328 107 38...,"[0.007298107258975506, 0.007080304622650147, 0..."
3,3|,623 328 649 582 488 12 578 623 538 382 382 265...,"[0.01401490531861782, 0.00508544840849936, 0.0..."
4,4|,83 293 398 797 382 363 145 424 693 698 800 691...,"[0.00969548262655735, 0.01638804133981466, 0.0..."


In [33]:
test_df[0]=test_df[0].apply(lambda x:x[:-1])
test_df.head()

,0,1,2
0,0,852 328 697 538 142 355 582 800 728 4 647 169 ...,"[0.012090771738439799, 0.005923829087987542, 0..."
1,1,380 358 343 654 171 832 47 832 690 693 48 563 ...,"[0.0017855744576081634, 0.002484714472666383, ..."
2,2,751 335 834 582 717 583 585 693 623 328 107 38...,"[0.007298107258975506, 0.007080304622650147, 0..."
3,3,623 328 649 582 488 12 578 623 538 382 382 265...,"[0.01401490531861782, 0.00508544840849936, 0.0..."
4,4,83 293 398 797 382 363 145 424 693 698 800 691...,"[0.00969548262655735, 0.01638804133981466, 0.0..."


In [34]:
sub_id=test_df[0].values
pres_all = sub.tolist()
str_w=''
with open(f'{save_path}/submit_{oof_score}.csv','w') as f:
    for i in range(len(sub_id)):
        pres_fold = pres_all[i]
        pres_fold=[str(p) for p in pres_fold]
        prob = " ".join(pres_fold)
        str_w+=sub_id[i]+'|'+','+'|'+prob+'\n'
    str_w=str_w.strip('\n')
    f.write(str_w)


In [35]:
test_df.to_csv(f"{save_path}/oof_test_{score}.csv")
oof_train.to_csv(f"{save_path}/oof_train_{score}.csv")